In [56]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


In [57]:
import cv2
import mediapipe as mp 
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [58]:
help(mp_drawing.DrawingSpec)

Help on class DrawingSpec in module mediapipe.python.solutions.drawing_utils:

class DrawingSpec(builtins.object)
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |  
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  -------------------------------------------------

In [59]:
from IPython.display import Image

Image(url="https://i.imgur.com/3j8BPdc.png", width=300)

In [60]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.624233
y: -0.06940222
z: -2.2621484
visibility: 0.99504364
, x: 0.6758174
y: -0.15298714
z: -2.27644
visibility: 0.9904702
, x: 0.70133567
y: -0.152074
z: -2.2764807
visibility: 0.9876695
, x: 0.72669184
y: -0.15081716
z: -2.2774584
visibility: 0.99141896
, x: 0.6044727
y: -0.20607615
z: -2.2513652
visibility: 0.99185014
, x: 0.58412516
y: -0.200037
z: -2.2518013
visibility: 0.9891264
, x: 0.56256425
y: -0.19329418
z: -2.2525964
visibility: 0.99206746
, x: 0.7416928
y: -0.14312445
z: -1.9742063
visibility: 0.98815024
, x: 0.5492094
y: -0.17433524
z: -1.8307798
visibility: 0.9881747
, x: 0.656288
y: -0.0049624443
z: -2.1172955
visibility: 0.9978079
, x: 0.5823208
y: -0.0051382184
z: -2.0759542
visibility: 0.9982262
, x: 0.8961749
y: 0.11892879
z: -1.6113746
visibility: 0.99627507
, x: 0.43767318
y: 0.13626546
z: -1.3321543
visibility: 0.9978668
, x: 1.1225324
y: 0.71418357
z: -1.4699165
visibility: 0.9612029
, x: 0.21511337
y: 0.61175233
z: -1.2581651
visibility: 0.9718478
, x: 1.

In [61]:
len(landmarks)

33

In [62]:
for lndmrk in mp_pose.PoseLandmark: 
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [63]:
landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

x: 0.40863872
y: 1.0662504
z: 0.06733455
visibility: 0.52002245

In [64]:
landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

x: 0.4600316
y: 1.5601028
z: 0.49629155
visibility: 0.074497156

In [65]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

x: 0.17926463
y: 0.5827183
z: -1.077434
visibility: 0.99469095

In [67]:
def calculate_deadlift_angle(right_shoulder, right_hip, right_knee):
    right_shoulder = np.array(right_shoulder)  
    right_hip = np.array(right_hip)  
    right_knee = np.array(right_knee)  
    
    # Calculate the angle between hip, knee, and shoulder
    radians = np.arctan2(right_knee[1] - right_hip[1], right_knee[0] - right_hip[0]) - np.arctan2(right_shoulder[1] - right_hip[1], right_shoulder[0] - right_hip[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle


In [68]:
right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
right_hip= [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
right_knee= [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

In [69]:
right_shoulder, right_hip, right_knee

([0.179264634847641, 0.5827183127403259],
 [0.40863871574401855, 1.0662504434585571],
 [0.4600315988063812, 1.5601028203964233])

In [70]:
calculate_deadlift_angle(right_shoulder, right_hip, right_knee)

160.56276513743154

In [71]:
# Convert landmark coordinates to pixel values
tuple(np.multiply(right_shoulder, [640, 480]).astype(int))
tuple(np.multiply(right_hip, [640, 480]).astype(int))
tuple(np.multiply(right_knee, [640, 480]).astype(int))


(294, 748)

In [72]:
cap = cv2.VideoCapture(0)

# Deadlift counter variables
counter = 0
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for deadlift landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate deadlift angle
            angle = calculate_deadlift_angle(right_shoulder, right_hip, right_knee)  # Modify function for deadlift angle
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(right_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Deadlift counter logic
            if angle > 170:
                stage = "up"
            if angle < 90 and stage == 'up':
                stage = "down"
                counter += 1
                print(counter)
                       
        except:
            pass
        
        # Render deadlift counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'DEADLIFTS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Deadlift Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [73]:
cap = cv2.VideoCapture(0)

# Deadlift counter variables
counter = 0
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            
            # Get coordinates for deadlift landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate deadlift angle
            angle = calculate_deadlift_angle(right_shoulder, right_hip, right_knee)  # Modify function for deadlift angle
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(right_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Deadlift counter logic
            # You need to define the specific logic for deadlift completion
            # For example, you might check for when the angle exceeds a certain threshold
            
            if angle > 170:
                stage = "up"
            if angle < 90 and stage == 'up':
                stage = "down"
                counter += 1
                
                       
        except:
            pass
        
        # Render deadlift counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'DEADLIFTS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, '    STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Deadlift Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
